In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import pickle
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

base_dir = 'c:/Users/deepa/Documents/Deeplearning CA/deep-learning-techniques-comparison-rnn-tcn'
preprocessed_dir = os.path.join(base_dir, 'preprocessed_data')

In [ ]:
with open(os.path.join(preprocessed_dir, 'bike_sharing.pkl'), 'rb') as f:
    bike_data = pickle.load(f)

X_train_seq = bike_data['X_train_seq'].astype(np.float32)
X_val_seq = bike_data['X_val_seq'].astype(np.float32)
X_test_seq = bike_data['X_test_seq'].astype(np.float32)

y_train_seq = bike_data['y_train_seq'].astype(np.float32)
y_val_seq = bike_data['y_val_seq'].astype(np.float32)
y_test_seq = bike_data['y_test_seq'].astype(np.float32)

print(f"X_train shape: {X_train_seq.shape}")
print(f"X_val shape: {X_val_seq.shape}")
print(f"X_test shape: {X_test_seq.shape}")
print(f"Sequence length: {X_train_seq.shape[1]}, Features: {X_train_seq.shape[2]}")

In [ ]:
seq_length = X_train_seq.shape[1]
num_features = X_train_seq.shape[2]

num_filters = 64
kernel_size = 3
epochs = 50
batch_size = 32
learning_rate = 0.001

model = Sequential([
    Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu', padding='same', input_shape=(seq_length, num_features)),
    Dropout(0.2),
    Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu', padding='same'),
    Dropout(0.2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

print(model.summary())

In [ ]:
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

In [ ]:
y_pred = model.predict(X_test_seq).flatten()

mse = mean_squared_error(y_test_seq, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_seq, y_pred)
r2 = r2_score(y_test_seq, y_pred)

print(f"\n{'='*50}")
print(f"TensorFlow TCN - Bike Sharing Regression Results")
print(f"{'='*50}")
print(f"MSE:  {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R²:   {r2:.4f}")

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(y_test_seq[:100], label='Actual', marker='o')
plt.plot(y_pred[:100], label='Predicted', marker='s')
plt.title('Actual vs Predicted (First 100 samples)')
plt.xlabel('Sample')
plt.ylabel('Bike Count')
plt.legend()
plt.show()